In [26]:
!pip install keras-tuner


In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import keras_tuner as kt

# Load dataset
df = pd.read_csv("/content/Alphabets_data.csv")


In [33]:

# Explore dataset
print(df.head())
print(df.info())
print(df.describe())


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbo

In [34]:
# Handle missing values
df.dropna(inplace=True)


In [35]:

# Separate features and target
X = df.drop(columns=['letter'])  # Assuming 'label' is the target column
y = df['letter']

In [36]:
# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [37]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [38]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [39]:
# Define ANN model
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                               activation=hp.Choice('activation', ['relu', 'tanh'])))
    model.add(layers.Dense(len(np.unique(y)), activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [0.001, 0.0001])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [41]:

# Hyperparameter tuning
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10,
                        executions_per_trial=1,
                        directory='hyperparameter_tuning',
                        project_name='ANN_tuning')

tuner.search(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32)


Trial 10 Complete [00h 00m 27s]
val_accuracy: 0.8681250214576721

Best val_accuracy So Far: 0.9237499833106995
Total elapsed time: 00h 07m 22s


In [42]:
# Get best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

In [43]:
# Train best model
history = best_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3666 - loss: 2.4022 - val_accuracy: 0.7234 - val_loss: 1.1173
Epoch 2/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7319 - loss: 1.0326 - val_accuracy: 0.7800 - val_loss: 0.8182
Epoch 3/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7904 - loss: 0.7632 - val_accuracy: 0.8147 - val_loss: 0.6921
Epoch 4/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8231 - loss: 0.6402 - val_accuracy: 0.8391 - val_loss: 0.5962
Epoch 5/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8512 - loss: 0.5363 - val_accuracy: 0.8472 - val_loss: 0.5413
Epoch 6/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8605 - loss: 0.4978 - val_accuracy: 0.8628 - val_loss: 0.4856
Epoch 7/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8791 - loss: 0.4448 - val_accuracy: 0.8747 - val_loss: 0.4508
Epoch 8/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8941 - loss: 0.3887 - val_accuracy: 0.

In [44]:
# Evaluate model
y_pred = np.argmax(best_model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test Accuracy: 0.9475
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       158
           1       0.94      0.88      0.91       153
           2       0.94      0.98      0.96       147
           3       0.96      0.93      0.94       161
           4       0.94      0.94      0.94       154
           5       0.92      0.95      0.93       155
           6       0.94      0.95      0.95       155
           7       0.87      0.88      0.88       147
           8       0.94      0.91      0.93       151
           9       0.97      0.93      0.95       149
          10       0.92      0.93      0.93       148
          11       0.96      0.93      0.95       152
          12       0.99      0.97      0.98       158
          13       0.95      0.93      0.94       157
          14       0.96      0.95      0.96       150
          15       0.95      0.97      0.96     